# Introduction to Commercial Optimization Solvers
## Mehmet Gönen
## October 24, 2022

The CPLEX Python API is a Python package named **cplex** that allows the Callable Library to be accessed from the Python programming language. It is equally suitable for interactive use through the Python interpreter or for writing scripts or full-fledged applications.

In [1]:
# load libraries
import numpy as np
import scipy.sparse as sp

import cplex as cp

## Solving an LP problem
\begin{align*}
\mbox{maximize} \;\;& 3x_{1} + 5x_{2} \\
\mbox{subject to:} \;\;& 1x_{1} + 0x_{2} \leq 4 \\
\;\;& 0x_{1} + 2x_{2} \leq 12 \\
\;\;& 3x_{1} + 2x_{2} \leq 18 \\
\;\;& x_{1} \geq 0 \\
\;\;& x_{2} \geq 0
\end{align*}

### Method 1

In [2]:
# create an empty optimization problem
prob1 = cp.Cplex()
prob1.set_problem_name("prob1")

# add decision variables to the problem including their coefficients in objective, ranges, and names
prob1.variables.add(obj = [3, 5], lb = [0, 0], ub = [cp.infinity, cp.infinity],
                    names = ["x1", "x2"])

# define problem type
prob1.objective.set_name("profit")
prob1.objective.set_sense(prob1.objective.sense.maximize)

# add constraints to the problem including their directions, right-hand side values, and names
prob1.linear_constraints.add(senses = ["L", "L", "L"],
                             rhs = [4, 12, 18],
                             names = ["c1", "c2", "c3"])

# add coefficients for each constraint
prob1.linear_constraints.set_coefficients(0, 0, 1)
prob1.linear_constraints.set_coefficients(1, 1, 2)
prob1.linear_constraints.set_coefficients(2, 0, 3)
prob1.linear_constraints.set_coefficients(2, 1, 2)

# display the problem
print(prob1.write_as_string())

# solve the problem
prob1.solve()

# check the solution status
print(prob1.solution.get_status())
print(prob1.solution.status[prob1.solution.get_status()])

# check the solution
x_star = prob1.solution.get_values()
obj_star = prob1.solution.get_objective_value()
print(x_star)
print(obj_star)

# print the problem to a file
prob1.write("prob1.lp")
prob1.write("prob1.mps")

\ENCODING=ISO-8859-1
\Problem name: prob1

Maximize
 profit: 3 x1 + 5 x2
Subject To
 c1: x1 <= 4
 c2: 2 x2 <= 12
 c3: 3 x1 + 2 x2 <= 18
End

Version identifier: 20.1.0.1 | 2022-01-05 | 9df5e5246
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 1 rows, 2 columns, and 2 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            36.000000
1
optimal
[2.0, 6.0]
36.0


In [3]:
# create an empty optimization problem
prob1_copy = cp.Cplex()

# read the problem from a file 
prob1_copy.read("prob1.lp")
prob1_copy.set_problem_name("prob1_copy")

# display the problem
print(prob1_copy.write_as_string())

# solve the problem
prob1_copy.solve()

# check the solution status
print(prob1_copy.solution.get_status())
print(prob1_copy.solution.status[prob1.solution.get_status()])

# check the solution
x_star = prob1_copy.solution.get_values()
obj_star = prob1_copy.solution.get_objective_value()
print(x_star)
print(obj_star)

\ENCODING=ISO-8859-1
\Problem name: prob1_copy

Maximize
 profit: 3 x1 + 5 x2
Subject To
 c1: x1 <= 4
 c2: 2 x2 <= 12
 c3: 3 x1 + 2 x2 <= 18
End

Version identifier: 20.1.0.1 | 2022-01-05 | 9df5e5246
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 1 rows, 2 columns, and 2 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            36.000000
1
optimal
[2.0, 6.0]
36.0


### Algebraic Representation
\begin{align*}
\mbox{maximize} \;\;& \boldsymbol{c}^{\top} \boldsymbol{x}\\
\mbox{subject to:} \;\;& \mathbf{A} \boldsymbol{x} \leq \boldsymbol{b} \\
\;\;& \boldsymbol{l} \leq \boldsymbol{x} \leq \boldsymbol{u}
\end{align*}

In [4]:
c = np.array([3, 5])
A = sp.csr_matrix(np.array([[1, 0],
                            [0, 2],
                            [3, 2]]))
senses = np.array(["L", "L", "L"])
b = np.array([4, 12, 18])
l = np.array([0, 0])
u = np.array([cp.infinity, cp.infinity])

variable_names = ["x{}".format(i + 1) for i in range(c.size)]
constraint_names = ["c{}".format(i + 1) for i in range(b.size)]

### Method 2

In [5]:
# create an empty optimization problem
prob2 = cp.Cplex()
prob2.set_problem_name("prob2")

# add decision variables to the problem including their coefficients in objective and ranges
prob2.variables.add(obj = c.tolist(), lb = l.tolist(), ub = u.tolist(),
                    names = variable_names)

# define problem type
prob2.objective.set_name("profit")
prob2.objective.set_sense(prob2.objective.sense.maximize)

# add constraints to the problem including their directions and right-hand side values
prob2.linear_constraints.add(senses = senses.tolist(),
                             rhs = b.tolist(),
                             names = constraint_names)

# add coefficients for each constraint
row_indices, col_indices = A.nonzero()
prob2.linear_constraints.set_coefficients(zip(row_indices.tolist(),
                                              col_indices.tolist(),
                                              A.data.tolist()))

# display the problem
print(prob2.write_as_string())

# solve the problem
prob2.solve()

# check the solution status
print(prob2.solution.get_status())
print(prob2.solution.status[prob2.solution.get_status()])

# check the solution
x_star = prob2.solution.get_values()
obj_star = prob2.solution.get_objective_value()
print(x_star)
print(obj_star)

\ENCODING=ISO-8859-1
\Problem name: prob2

Maximize
 profit: 3 x1 + 5 x2
Subject To
 c1: x1 <= 4
 c2: 2 x2 <= 12
 c3: 3 x1 + 2 x2 <= 18
End

Version identifier: 20.1.0.1 | 2022-01-05 | 9df5e5246
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 2 rows and 0 columns.
Reduced LP has 1 rows, 2 columns, and 2 nonzeros.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual objective     =            36.000000
1
optimal
[2.0, 6.0]
36.0
